### PEI Child Care
Retrieved from: https://peichildcareregistry.com/search.php

Robots.txt: Not found.

`java -jar /usr/local/bin/selenium-server-standalone-3.141.59.jar`

In [34]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import random
from tqdm import tqdm_notebook as tqdmn
import folium
from logger import logging

In [35]:
pd.set_option('display.max_columns', 60)

In [36]:
opts = Options()
executer = "user-agent=kaitlyn hobbs bot version 1.0, kaitlyn.hobbs@statcan.gc.ca"
opts.add_argument(executer)
driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)


Looking for [chromedriver 86.0.4240.22 mac64] driver in cache 
File found in cache by path [/Users/kt/.wdm/drivers/chromedriver/86.0.4240.22/mac64/chromedriver]


#### Execution log:

In [ ]:
FORMAT = "%(asctime)-15s %(clientip)s %(user)-8s %(message)s"
logging.basicConfig(filename="logfilename.log", format=FORMAT)
logging.info('Execution of MB child care web scrape. \n{}.'.format(executer))

#### Execute Scrape:

In [37]:
url = 'https://peichildcareregistry.com/search.php'
driver.get(url)

In [39]:
pe = {}
for i in range(1,9):
    # page iteration
    driver.find_element_by_xpath('/html/body/main/section/div/div/div[2]/div/div[4]/a[{}]'.format(i)).click()
    print("Page {}:".format(i))
    if i in range(8):
        # ie. not the last page
        for num in range(1,11):
            print("{} of 10".format(num))
            box = driver.find_element_by_xpath('/html/body/main/section/div/div/div[2]/div/div[3]/div/div/div/div/div[{}]/div[1]/div[1]'.format(num))
            name = box.find_element_by_tag_name('h2').text
            # get rest of information in the childcare facility box
            elemnt = box.find_element_by_xpath('/html/body/main/section/div/div/div[2]/div/div[3]/div/div/div/div/div[{}]/div[1]/div[1][text()]'.format(num)).text.split('\n')

            # set up name in pe dictionary
            if 'Name' not in pe.keys():
                pe['Name'] = [name]
            else:
                pe['Name'].append(name)

            # add the rest of the facility info to dictionary
            for i in elemnt[1::]:
                try:
                    if i.split(': ')[0] not in pe.keys():
                        pe[i.split(': ')[0]] = [i.split(': ')[1]]
                    else:
                        pe[i.split(': ')[0]].append(i.split(': ')[1])
                except IndexError:
                    pass
                
        # wait between pages
        time.sleep(random.uniform(5,15))

    if i == 8:
        # last page
        for num in range(1,8):
            print("{} of 7".format(num))
            box = driver.find_element_by_xpath('/html/body/main/section/div/div/div[2]/div/div[3]/div/div/div/div/div[{}]/div[1]/div[1]'.format(num))
            name = box.find_element_by_tag_name('h2').text
            # get rest of information in the childcare facility box
            elemnt = box.find_element_by_xpath('/html/body/main/section/div/div/div[2]/div/div[3]/div/div/div/div/div[{}]/div[1]/div[1][text()]'.format(num)).text.split('\n')

            # add the rest of the facility info to dictionary
            pe['Name'].append(name)
            
            for i in elemnt[1::]:
                try:
                    if i.split(': ')[0] not in pe.keys():
                        pe[i.split(': ')[0]] = [i.split(': ')[1]]
                    else:
                        pe[i.split(': ')[0]].append(i.split(': ')[1])
                except IndexError:
                    pass
                

# last page
# '/html/body/main/section/div/div/div[2]/div/div[3]/div/div/div/div/div[7]'

Page 1:
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10
Page 2:
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10
Page 3:
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10
Page 4:
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10
Page 5:
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10
Page 6:
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10
Page 7:
1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10
Page 8:
1 of 7
2 of 7
3 of 7
4 of 7
5 of 7
6 of 7
7 of 7


In [46]:
driver.close()

#### Clean & Export:

In [40]:
for k,v in pe.items():
    print(k, len(v))

Name 77
Address 77
Operation Hours 77
Centre Accepts 77
Contact Name 77
E-mail 77
Phone 77
Centre Accommodates 57


In [41]:
# Drop unneccessary key with missing values (Centre Accommdates gives hours: part time, seasonal, alternate)
pe.pop("Centre Accommodates")

['Part Time',
 'Part Time',
 'Part Time, Seasonal, Alternate Hours',
 'Part Time',
 'Part Time',
 'Part Time, Alternate Hours',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time, Francophone',
 'Part Time, Seasonal, Francophone',
 'Part Time',
 'Part Time',
 'Part Time, Seasonal',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time, Seasonal, Alternate Hours',
 'Francophone',
 'Part Time, Alternate Hours',
 'Seasonal',
 'Part Time',
 'Part Time, Seasonal',
 'Part Time',
 'Francophone',
 'Part Time, Francophone',
 'Francophone',
 'Part Time',
 'Part Time, Alternate Hours',
 'Part Time',
 'Part Time',
 'Part Time, Seasonal',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time, Seasonal, Alternate Hours',
 'Part Time',
 'Part Time, Alternate Hours',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time',
 'Part Time

In [45]:
pd.DataFrame(pe).to_csv('data/childcare/PE-childcare.csv')